In [ ]:
!python3 -m pip install "pymongo[srv]"
!pip install dnspython

# Data extraction

In [ ]:
from pymongo import MongoClient
from urllib.parse import quote_plus
import pandas as pd
# pip3 install pymongo[srv]


def data_extract():
    host = "2kbot.dp2to.mongodb.net/users-linkedin-scraped?retryWrites=true&ssl=true&ssl_cert_reqs=CERT_NONE"

    user = 'users-linkedin-scraped'
    password = 'IsVSEzCPGhUJFQJe'
    uri = "mongodb+srv://%s:%s@%s" % (quote_plus(user), quote_plus(password), host)
    client = MongoClient(uri)

    db = client.get_database('users-linkedin-scraped')
    #print(db.list_collection_names())
    collection_employees = db.get_collection("users").find()
    list_cur = list(collection_employees)
    df_raw = pd.DataFrame(list_cur)
    # print(df_raw.head())
    #print(df_raw.columns)
    return df_raw


def companies_data_extract():
    host = "2kbot.dp2to.mongodb.net/users-linkedin-scraped?retryWrites=true&ssl=true&ssl_cert_reqs=CERT_NONE"

    user = 'users-linkedin-scraped'
    password = 'IsVSEzCPGhUJFQJe'
    uri = "mongodb+srv://%s:%s@%s" % (quote_plus(user), quote_plus(password), host)
    client = MongoClient(uri)

    db = client.get_database('users-linkedin-scraped')

    collection_employees = db.get_collection("companies").find()
    list_cur = list(collection_employees)
    df_raw = pd.DataFrame(list_cur)
    return df_raw


# Data transformation


## Companies transformation

In [ ]:
import logging
from keras.utils.np_utils import to_categorical

def cat_company_size(siz):
    try:
        resp = None
        if siz:
            size_dict = {0: [1, None],
                         1: [1, 10],
                         2: [11, 50],
                         3: [51, 200],
                         4: [201, 500],
                         5: [501, 1000],
                         6: [1001, 5000],
                         7: [5001, 10000],
                         8: [10001, None]}
            for key, value in size_dict.items():
                if siz == value:
                    resp = key
        else:
            resp = 0
        if not resp:
            resp = 0
        return resp
    except:
        logging.exception("Error in companies: ")


def homogen_companies(company):
    company_cln = company.lower().replace("é", "e").replace("è", "e").replace("à", "a").replace(",", "") \
        .replace("ó", "o").replace("á", "a").replace(".", "").replace("-", "").replace("(", "").replace(")", "") \
        .replace("'", "").replace(" ", "").replace("espa%c3%b1a", "")
    return company_cln


def check_freelancer(job, size_cat):
    if job and any(x in job for x in ["freelance", "self employed", "self-employed", "autonomo", "freelancing"]):
        return 1
    else:
        return size_cat


def find_company(experiencia, i):
    try:
        comp = homogen_companies(experiencia[i]['company'])
    except:
        comp = None
    return comp


def search_comp_name_in_comp(lista_comp, comp):
    if comp:
        for c in lista_comp:
            if comp in c:
                comp = c
                break
    return comp


def search_comp_name_in_user(companies, comp):
    if comp:
        for c in companies:
            if c in comp and len(c) > 3:
                comp = c
                break
    return comp


def company_data():
    df_raw = companies_data_extract()
    df_cln = df_raw[['industry', 'company_size', 'locations', 'name', 'universal_name_id', 'similar_companies']]

    industries = df_cln.industry.unique().tolist()
    i = 0
    dict_indus = {}
    dict_indus_type = {}
    l = []
    l.extend(range(0, len(industries)+1))
    a = to_categorical(l, num_classes=len(industries)+1)
    for industry in industries:
        dict_indus[industry] = i
        dict_indus_type[i] = a[i]
        i += 1

    df_cln['industries_cat'] = df_cln.apply(lambda row: dict_indus_type[dict_indus[row.industry]], axis=1)

    df_cln['size_cat'] = df_cln.apply(lambda row: cat_company_size(row.company_size), axis=1)
    df_cln.loc[df_cln.universal_name_id == '', 'universal_name_id'] = df_cln['name']
    df_cln.loc[df_cln.universal_name_id == 'ing', 'universal_name_id'] = "ing direct"
    df_cln.universal_name_id.fillna(df_cln.name, inplace=True)
    df_cln['universal_name_id'] = df_cln.apply(lambda row: homogen_companies(row.universal_name_id), axis=1)

    return df_cln


def merge_comp(df1, df2, i):
    dfm = df1.merge(df2, on=f'job_{i}_company', how='left', suffixes=('', f'_{i}'))
    return dfm


def company_enrichment(df, df_comp, i, dict_size):
    companies = df_comp.universal_name_id.unique().tolist()
    df_comp = df_comp.rename(columns={'universal_name_id': f'job_{i}_company', 'size_cat': f'size_cat{i}'})
    df[f'job_{i}_company'] = df.apply(lambda row: find_company(row.experiences, i - 1), axis=1)
    # primer intento: a lo burro
    df_raw_cols = list(df)
    df_enrich = merge_comp(df, df_comp, i).drop_duplicates(subset=['_id'])
    df_enrich[f'size_cat{i}'] = df_enrich.apply(lambda row: check_freelancer(row[f'job_{i}_company'], row[f'size_cat{i}']),
                                            axis=1)
    df_temp = df_enrich.loc[df_enrich[f'size_cat{i}'].isnull()]
    df_temp = df_temp[df_raw_cols]
    df_enrich = df_enrich.loc[~df_enrich[f'size_cat{i}'].isnull()]
    df_temp[f'job_{i}_company'] = df_temp.apply(
        lambda row: search_comp_name_in_comp(companies, row[f'job_{i}_company']), axis=1)
    df_temp = merge_comp(df_temp, df_comp, i)
    df_enrich = df_enrich.append(df_temp).drop_duplicates(subset=['_id'])
    df_temp = df_enrich.loc[df_enrich[f'size_cat{i}'].isnull()]
    df_temp = df_temp[df_raw_cols]
    df_enrich = df_enrich.loc[~df_enrich[f'size_cat{i}'].isnull()]
    df_temp[f'job_{i}_company'] = df_temp.apply(
        lambda row: search_comp_name_in_user(companies, row[f'job_{i}_company']), axis=1)
    df_temp = merge_comp(df_temp, df_comp, i)
    df_enrich = df_enrich.append(df_temp).drop_duplicates(subset=['_id'])
    df_enrich.loc[df_enrich[f'size_cat{i}'].isnull(), f'size_cat{i}'] = 9
    df_enrich[f'size_cat{i}_class'] = df_enrich.apply(lambda row: dict_size[int(row[f'size_cat{i}'])], axis=1)
    return df_enrich



## Jobs recognition

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import json
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import SnowballStemmer

global words_set
nltk.download('perluniprops')
nltk.download('stopwords')
pd.options.mode.chained_assignment = None  # default='warn'


def get_training_data():
    global words_set
    words_set = []
    with open('drive/MyDrive/databases/dict_num.json') as json_file:
        dict_num = json.load(json_file)
    init_df = pd.read_csv('drive/MyDrive/databases/init_df.csv')
    with open('drive/MyDrive/databases/job_cat.json') as json_file:
        data = json.load(json_file)
    next_df = pd.DataFrame({'Titulo': data.keys(), 'OHE_Label': data.values()})

    one_hot_encoding = []
    final_shape = max(init_df['OHE_Label'].to_list())
    for i in init_df['OHE_Label']:
        vector = [0] * final_shape
        vector[i - 1] = 1
        one_hot_encoding.append(vector)
    init_df['OHE_Label'] = one_hot_encoding

    one_hot_encoding = []
    final_shape = max(next_df['OHE_Label'].to_list())
    for i in next_df['OHE_Label']:
        vector = [0] * final_shape
        vector[i - 1] = 1
        one_hot_encoding.append(vector)
    next_df['OHE_Label'] = one_hot_encoding
    next_df['BoW'] = [None] * next_df.shape[0]

    tercer_df = pd.read_csv('drive/MyDrive/databases/tercer_dataset.csv', sep=';')

    tercer_df['Codigo'] = [math.trunc(x / 100) for x in tercer_df['Codigo']]
    tercer_df = tercer_df[tercer_df['Codigo'] > 0]
    tercer_df['OHE_Label'] = [dict_num[str(x)] for x in tercer_df['Codigo']]

    one_hot_encoding = []
    final_shape = max(tercer_df['OHE_Label'].to_list())
    for i in tercer_df['OHE_Label']:
        vector = [0] * final_shape
        vector[i - 1] = 1
        one_hot_encoding.append(vector)
    tercer_df['OHE_Label'] = one_hot_encoding
    tercer_df['BoW'] = [None] * tercer_df.shape[0]
    tercer_df = tercer_df[['Titulo', 'BoW', 'OHE_Label']]

    df = init_df.append(next_df, ignore_index=True)
    df = df.append(tercer_df, ignore_index=True)
    return df


def remove_stopwords(palabras):
    spanish_stemmer = SnowballStemmer('spanish')
    global words_set

    sotpwords_set = list(set(stopwords.words('spanish')))
    sotpwords_set.append('(')
    sotpwords_set.append(')')
    sotpwords_set.append('[')
    sotpwords_set.append(']')
    sotpwords_set.append(';')
    sotpwords_set.append(',')
    sotpwords_set.append(':')
    final_text = []

    for word in palabras:
        if word not in sotpwords_set:
            final_word = word.lower()
            index_dot = final_word.find('.')
            if index_dot != -1:
                final_word = final_word[:index_dot]
            index_dot = final_word.find(',')
            if index_dot != -1:
                final_word = final_word[:index_dot]
            index_dot = final_word.find('-')
            if index_dot != -1:
                final_word = final_word[:index_dot]
            final_word = spanish_stemmer.stem(final_word)
            words_set.append(final_word)
            words_set = list(set(words_set))
            final_text.append(final_word)

    return final_text


def create_model(df_train):

    toktok = ToktokTokenizer()
    textos_tokenizados = []

    for text in df_train['Titulo']:
        tokens = toktok.tokenize(text)
        tokens = remove_stopwords(tokens)
        textos_tokenizados.append(tokens)
    global words_set
    dictionary = {}
    count = 1
    for i in words_set:
        dictionary[i] = count
        count += 1

    bow_corpus = []
    n = len(words_set)
    for i in textos_tokenizados:
        bow_text = [0] * n
        for j in i:
            bow_text[dictionary[j] - 1] = 1
        bow_corpus.append(bow_text)
    df_train['BoW'] = bow_corpus

    x = df_train['BoW'].to_list()
    y = df_train['OHE_Label'].to_list()

    init_shape = len(x[0])
    final_shape = len(y[0])
    print(f"init: {init_shape}")
    model = keras.Sequential()
    model.add(layers.Dense(init_shape, input_dim=init_shape, activation='relu'))
    model.add(layers.Dense(1500, activation='relu'))
    model.add(layers.Dense(1000, activation='relu'))
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(final_shape, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(),
                  metrics=['accuracy'])

    model.fit(x, y, epochs=20, batch_size=64, verbose=0)
    return model


def preprocess_text(text):
    toktok = ToktokTokenizer()
    global words_set
    tokens = toktok.tokenize(text)
    tokens = remove_stopwords(tokens)
    n = len(words_set)
    bow_text = [0] * n
    dictionary = {}
    count = 1
    for i in words_set:
        dictionary[i] = count
        count += 1

    for j in tokens:
        try:
            bow_text[dictionary[j] - 1] = 1
        except:
            pass
            # print('La palabra', j, 'no esta en el diccionario.')
    return bow_text


def pred_jobs(model, df_raw, ind):
    bow = df_raw[f'BoW_{ind}'].to_list()
    # for i in bow:
    #    print(len(i))
    pred = model.predict(bow)
    # print(pred)

    pred_num = []
    for ind in pred:
        max_index = np.argmax(ind) + 1
        pred_num.append(max_index)
    df_raw[f'job_{ind}_category'] = pred_num
    return df_raw[f'job_{ind}_category']


def jobs_prediction(df):
    print("\n Starting jobs prediction...")
    with open('drive/MyDrive/databases/dict_change.json') as file:
        dict_change = json.load(file)
    df_train = get_training_data()
    for i in range(1, 7):
        df_temp = df[[f'job_{i}_category', f'job_{i}']]
        df_temp[f'BoW_{i}'] = df_temp[f'job_{i}'].apply(preprocess_text)
    del df_temp

    try:
        model = keras.models.load_model("drive/MyDrive/databases/jobs_prediction")
        print("model loaded")
    except:
      try:
        model = keras.models.load_model("drive/MyDrive/databases/jobs_prediction")
        print("model loaded")
      except:
        print("creating model...")
        model = create_model(df_train)
        print("model created")
        model.save('jobs_prediction')

    for i in range(1, 7):
        null_df = df[[f'job_{i}_category', f'job_{i}']].loc[df[f'job_{i}_category'].isnull()]
        null_df[f'BoW_{i}'] = null_df[f'job_{i}'].apply(preprocess_text)
        null_df[f'job_{i}_category'] = pred_jobs(model, null_df, i)
        df.loc[df[f'job_{i}'].isin(null_df[f'job_{i}']), [f'job_{i}_category']] = null_df[
            [f'job_{i}_category', f'job_{i}']]

        df[f'job_{i}_flag'] = df.apply \
            (lambda row: dict_change[str(row[f'job_{i}_category']).split('.')[0]] if row[f'job_{i}'] != 'sin experiencia' else None,
             axis=1)
        print(f"jobs {i} predicted")
    if 'Unnamed: 0' in df.columns:
        df.drop('Unnamed: 0', axis=1)
    print(df.shape)
    return df


[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## main transformation

In [ ]:
import pandas as pd
import json
import time
from keras.utils.np_utils import to_categorical

pd.options.mode.chained_assignment = None


def find_clave(puesto, palabras_clave):
    if puesto == 'sin experiencia':
        return puesto
    else:
        for palabra in palabras_clave:
            puesto = puesto.lower().replace('ó', 'o').replace('é', 'e').replace('í', 'i').replace('á', 'a')
            if palabra.lower() in puesto:
                return palabra


def find_category(clave, categorias):
    if clave:
        if clave == 'sin experiencia':
            cat_out = 0
        else:
            cat_out = categorias[clave]
    else:
        cat_out = None
    return cat_out


def find_job(experiencia, i):
    try:
        job = experiencia[i]['title'].lower()
    except:
        job = 'sin experiencia'
    return job


def find_duration(experiencia, i):
    try:
        year = experiencia[i]['starts_at']['year']
        if not experiencia[i]['starts_at']['month']:
            month = 0
        else:
            month = experiencia[i]['starts_at']['month']
        if not experiencia[i]['ends_at']:
            year_fin = time.gmtime().tm_year
            month_fin = time.gmtime().tm_mon
        else:
            year_fin = experiencia[i]['ends_at']['year']
            if not experiencia[i]['ends_at']['month']:
                month_fin = 0
            else:
                month_fin = experiencia[i]['ends_at']['month']
        duration = (year_fin - year) * 12 + (month_fin - month)

    except:
        duration = 0

    return duration


def clasif_lang(len_dict, lista_leng, cat_dict):
    lista_clasif = []
    if lista_leng and type(lista_leng) is not float:
        try:
            for i in lista_leng:
                i = i.lower().split()[0].split("-")[0].replace('é', 'e').replace('á', 'a').replace('ñ', 'Ã±').replace('ç',
                                                                                                                      'Ã§') \
                    .replace("ê", "e").replace("è", "e").replace("à", "a").replace(",", "").replace("ö", "o")
                if i in len_dict:
                    lista_clasif.append(cat_dict[len_dict[i]])
        except:
            print(lista_leng)
            raise
    return lista_clasif


def certis(certs):
    resp = []
    for i in certs:
        if i:
            resp.append(i['name'])
    if resp:
        return resp
    else:
        return None


def clean_string(string):
    string = string.lower()
    string = string.replace('ó', 'o').replace('é', 'e').replace('í', 'i').replace('á', 'a').replace('ú', 'u')
    return string


def data_transformation(df):
    palabras_clave_raw = pd.read_excel('drive/MyDrive/databases/palabras_clave_jobs.xlsx', engine='openpyxl').drop('Unnamed: 1',
                                                                                                         axis=1)
    palabras_clave = palabras_clave_raw.palabras_clave.tolist()
    with open('drive/MyDrive/databases/job_cat.json') as f:
        cat_palabras = json.load(f)
    with open('drive/MyDrive/databases/idiomas.json', encoding='latin-1') as f:
        idiomas_dict = json.load(f)
        # print(idiomas_dict)
    df_comp = company_data()
    df_cleaned = df[['_id', 'occupation', 'experiences', 'education', 'languages', 'accomplishment_courses',
                     'certifications', 'headline', 'country', 'city', 'state']]

    l = []
    d = {}
    l.extend(range(1, 31))
    a = to_categorical(l, num_classes=31)
    # para categorizar los idiomas
    for k in range(1, 31):
        d[k] = a[k - 1]

    df_cleaned['languages'] = df_cleaned.apply(lambda row: clasif_lang(idiomas_dict, row.languages, d), axis=1)
    df_idiomas = pd.DataFrame(df_cleaned.languages.values.tolist()).add_prefix('language_')
    df_cleaned = pd.concat([df_cleaned, df_idiomas], axis=1)

    # Limpieza y clasificación de puestos de trabajo
    l = []
    d_i = {}
    l.extend(range(0, 11))
    a = to_categorical(l, num_classes=11)
    # para categorizar los tamaños de empresa
    for j in range(0, 11):
        d_i[j] = a[j]

    for i in range(1, 7):
        df_cleaned[f'job_{i}'] = df_cleaned.apply(lambda row: find_job(row.experiences, i - 1), axis=1)
        df_cleaned[f'job_{i}_flag'] = df_cleaned.apply(lambda row: find_clave(row[f'job_{i}'], palabras_clave), axis=1)
        df_cleaned[f'job_{i}_category'] = df_cleaned.apply(
            lambda row: find_category(row[f'job_{i}_flag'], cat_palabras), axis=1)
        df_cleaned[f'job_{i}_duration'] = df_cleaned.apply(lambda row: find_duration(row.experiences, i - 1), axis=1)
        df_cleaned = company_enrichment(df_cleaned, df_comp, i, d_i)

    print(df_cleaned.shape)
    # limpieza educacion
    df_cleaned['estudios'] = df_cleaned.apply(
        lambda row: row.education[0]['degree_name'] if row.education and type(row.education) == list else None,
        axis=1)
    df_cleaned['universidad'] = df_cleaned.apply(
        lambda row: row.education[0]['school'] if row.education and type(row.education) == list else None,
        axis=1)
    df_cleaned['certifications'] = df_cleaned.apply(
        lambda row: certis(row.certifications) if type(row.certifications) == list else None, axis=1)
    df_cleaned['accomplishment_courses'] = df_cleaned.apply(
        lambda row: certis(row.accomplishment_courses) if type(row.certifications) == list else None, axis=1)
    df_cleaned = df_cleaned.drop(['experiences', "education"], axis=1)
    df_edited = df_cleaned.loc[df_cleaned['job_1'] != 'Sin experiencia']
    # print(df_edited[['job_1', 'job_1_flag', 'job_1_category']])
    incogs = df_cleaned.loc[df_cleaned['job_1_category'].isnull()]
    # print(incogs[['job_1']].head(10))

    print(f"\nNº de puestos no clasificados: {incogs.shape[0]} puestos")
    print(f"porcentaje del total: {round(100 * (incogs.shape[0] / df_edited.shape[0]), 1)}%")

    df_edited_2 = jobs_prediction(df_edited)
    print("jobs predicted, finishing categorization...")
    lista_trabajos = df_edited_2['job_1_category'].unique().tolist()
    dict_cat = {}
    l = []
    l.extend(range(0, len(lista_trabajos) + 1))
    i = 0
    a = to_categorical(l, num_classes=len(l))
    for trab in lista_trabajos:
        dict_cat[trab] = a[i]
        i += 1

    for i in range(1, 7):
        df_cleaned[f'job_{i}_class'] = df_cleaned.apply(
            lambda row: dict_cat[row[f'job_{i}_category']] if not row[f'job_{i}_category'] or type(
                row[f'job_{i}_category'])   != float else None, axis=1)
    df_edited_2 = df_edited_2.drop_duplicates(subset=['_id'])
    return df_edited_2


# main

In [ ]:
import os


# pip3 install openpyxl (para exportar a excel)

if __name__ == '__main__':
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'    # quitar los logs de tensor
    # create_jobs_table()
    df1 = data_extract()
    df2 = data_transformation(df1)

    #df2.to_excel("data_cleaned2.xlsx")
    df2.to_csv("data_cleaned2.csv")
    print('Proceso finalizado, tenga un buen día')


(12369, 98)

Nº de puestos no clasificados: 3794 puestos
porcentaje del total: 30.7%

 Starting jobs prediction...
creating model...
init: 9339
model created
INFO:tensorflow:Assets written to: jobs_prediction/assets
jobs 1 predicted
jobs 2 predicted
jobs 3 predicted
jobs 4 predicted
jobs 5 predicted
jobs 6 predicted
(12369, 98)
jobs predicted, finishing categorization...
Proceso finalizado, tenga un buen día


# Data export

In [ ]:
#records = json.loads(df2.T.to_json()).values()
#db.myCollection.insert_many(records)